<a href="https://colab.research.google.com/github/wiiiiish/2022-machine_learning/blob/main/lstm_model_0512.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!ls

drive  sample_data


In [2]:
%cd drive/MyDrive/Colab\ Notebooks/2022-05-12\ lstm

/content/drive/MyDrive/Colab Notebooks/2022-05-12 lstm


In [3]:
# import library
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle

from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv1D, MaxPool1D
from tensorflow.keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

%matplotlib inline
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)

# Set seeds to make the experiment more reprodecible
from tensorflow.compat.v1 import set_random_seed
from numpy.random import seed
set_random_seed(1)
seed(1)

In [4]:
!unzip ./data/data.zip -d ./data

Archive:  ./data/data.zip
replace ./data/y_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ./data/y_train.csv      
replace ./data/y_test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ./data/y_test.csv       
replace ./data/X_test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ./data/X_test.csv       
replace ./data/X_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ./data/X_train.csv      


In [5]:
test_X = pd.read_csv('./data/X_test.csv', encoding='utf-8')
test_y = pd.read_csv('./data/y_test.csv', encoding='utf-8')
train_X = pd.read_csv('./data/X_train.csv', encoding='utf-8')
train_y = pd.read_csv('./data/y_train.csv', encoding='utf-8')

In [6]:
len(train_X)

34201

In [7]:
len(train_y)

34201

In [8]:
train_X = train_X.drop(train_X.columns[0], axis=1)
train_X.iloc[360, :]

date_time           05-06 02:00:00
temperature              19.619841
precipitation                  0.0
wind_speed                0.640395
humidity                 98.912399
radiation                      0.0
total_cloudiness          7.415161
visibility              653.974524
ground_temp              20.127291
now_aod                        0.0
before_hour_aod                0.0
Name: 360, dtype: object

In [9]:
train_X.iloc[34200,:]

date_time           05-10 00:00:00
temperature              23.495307
precipitation                  0.0
wind_speed                1.308598
humidity                 93.507282
radiation                      0.0
total_cloudiness          2.700084
visibility              841.329172
ground_temp              25.617869
now_aod                        0.0
before_hour_aod                0.0
Name: 34200, dtype: object

In [10]:
train_y = train_y.drop(train_y.columns[0], axis=1)
train_y

,aod
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
34196,0.0
34197,0.0
34198,0.0
34199,0.0


In [11]:
X = train_X.loc[:, 'temperature':]
X

,temperature,precipitation,wind_speed,humidity,radiation,total_cloudiness,visibility,ground_temp,now_aod,before_hour_aod
0,19.864374,0.0,0.822489,98.925436,0.0,7.547328,743.262485,20.429062,0.0,0.0
1,19.908106,0.0,0.883391,99.091616,0.0,6.534823,665.751451,20.591084,0.0,0.0
2,20.027289,0.0,0.828924,99.056635,0.0,6.820024,626.004984,20.365159,0.0,0.0
3,20.214164,0.0,0.895508,99.074766,0.0,7.301961,658.978883,20.482513,0.0,0.0
4,19.815515,0.0,0.814909,98.971507,0.0,7.149459,705.821344,20.624097,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
34196,24.073217,0.0,1.284951,89.005958,0.0,4.471558,882.632247,26.128100,0.0,0.0
34197,24.181822,0.0,1.317443,91.923205,0.0,2.536581,905.367195,26.246212,0.0,0.0
34198,23.765830,0.0,1.350209,89.787909,0.0,7.882563,908.718558,26.224739,0.0,0.0
34199,24.013697,0.0,1.417440,90.933227,0.0,2.849746,874.759202,26.427768,0.0,0.0


In [12]:
X = np.array(X)
X

array([[19.86437382,  0.        ,  0.82248939, ..., 20.42906155,
         0.        ,  0.        ],
       [19.90810568,  0.        ,  0.88339129, ..., 20.59108412,
         0.        ,  0.        ],
       [20.02728917,  0.        ,  0.82892439, ..., 20.36515904,
         0.        ,  0.        ],
       ...,
       [23.76582951,  0.        ,  1.35020924, ..., 26.22473944,
         0.        ,  0.        ],
       [24.01369698,  0.        ,  1.41743983, ..., 26.42776813,
         0.        ,  0.        ],
       [23.4953066 ,  0.        ,  1.3085976 , ..., 25.61786865,
         0.        ,  0.        ]])

In [13]:
X.shape

(34201, 10)

In [14]:
temp = []
for i in range(len(X)-360):
  for j in range(360):
    temp.append(X[i+j])
train_data = np.array(temp)
train_data.shape

(12182760, 10)

In [15]:
X_train = train_data.reshape(33841,360,10)
X_train.shape

(33841, 360, 10)

In [16]:
y = train_y.iloc[360:, :]
y

,aod
360,0.0
361,0.0
362,0.0
363,0.0
364,0.0
...,...
34196,0.0
34197,0.0
34198,0.0
34199,0.0


In [17]:
y_train = np.array(y)
print(y_train.shape)
y_train

(33841, 1)


array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [ ]:
adam = tf.keras.optimizers.Adam(learning_rate = 0.0001)
SGD = tf.keras.optimizers.SGD(learning_rate = 0.0001)
rmsprop = tf.keras.optimizers.RMSprop(learning_rate = 0.0001)

In [39]:
model_lstm = Sequential()
model_lstm.add(LSTM(50, activation='tanh', input_shape=(360, 10)))
model_lstm.add(Dense(1))
model_lstm.compile(loss='mae', optimizer='Adam', metrics = ['mae'])

In [40]:
model_lstm.fit(X_train,y_train,epochs = 30)

Epoch 1/30
1058/1058 [==============================] - 172s 134ms/step - loss: 134.7500 - mae: 134.7500
Epoch 2/30
1058/1058 [==============================] - 141s 134ms/step - loss: 128.8141 - mae: 128.8141
Epoch 3/30
1058/1058 [==============================] - 140s 132ms/step - loss: 122.6854 - mae: 122.6854
Epoch 4/30
1058/1058 [==============================] - 139s 132ms/step - loss: 118.0265 - mae: 118.0265
Epoch 5/30
1058/1058 [==============================] - 141s 133ms/step - loss: 111.6099 - mae: 111.6099
Epoch 6/30
1058/1058 [==============================] - 141s 133ms/step - loss: 106.1702 - mae: 106.1702
Epoch 7/30
1058/1058 [==============================] - 146s 138ms/step - loss: 101.1418 - mae: 101.1418
Epoch 8/30
1058/1058 [==============================] - 142s 134ms/step - loss: 96.4436 - mae: 96.4436
Epoch 9/30
1058/1058 [==============================] - 144s 136ms/step - loss: 91.9512 - mae: 91.9512
Epoch 10/30
1058/1058 [==============================] - 14

In [44]:
model_lstm.save('./lstm_model.h5')

In [41]:
from tensorflow.keras.layers import SimpleRNN

In [42]:
model_rnn = Sequential()
model_rnn.add(SimpleRNN(50, input_shape = (360,10)))
model_rnn.add(Dense(1))
model_rnn.compile(loss='mae', optimizer='Adam', metrics = ['mae'])

In [ ]:
# 1. optimizer 수정, 값 변화
## adam learning_rate 기본값 : 0.001

# 2. optimizer 변경 rmsprop / SGD

In [45]:
model_rnn.fit(X_train,y_train,epochs = 30)

Epoch 1/30
1058/1058 [==============================] - 69s 65ms/step - loss: 126.2892 - mae: 126.2892
Epoch 2/30
1058/1058 [==============================] - 63s 59ms/step - loss: 115.6992 - mae: 115.6992
Epoch 3/30
1058/1058 [==============================] - 63s 59ms/step - loss: 106.3968 - mae: 106.3968
Epoch 4/30
1058/1058 [==============================] - 62s 58ms/step - loss: 98.0005 - mae: 98.0005
Epoch 5/30
1058/1058 [==============================] - 62s 59ms/step - loss: 91.6076 - mae: 91.6076
Epoch 6/30
1058/1058 [==============================] - 62s 59ms/step - loss: 84.8259 - mae: 84.8259
Epoch 7/30
1058/1058 [==============================] - 62s 59ms/step - loss: 78.5032 - mae: 78.5032
Epoch 8/30
1058/1058 [==============================] - 63s 59ms/step - loss: 72.4183 - mae: 72.4183
Epoch 9/30
1058/1058 [==============================] - 64s 60ms/step - loss: 73.1874 - mae: 73.1874
Epoch 10/30
1058/1058 [==============================] - 62s 58ms/step - loss: 65.862

In [46]:
model_rnn.save('./rnn_model.h5')